In [244]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [245]:
data = pd.read_csv("../dataset/iris.csv")

data.loc[data['Species']=='Iris-setosa', 'Species'] = 0
data.loc[data['Species']=='Iris-versicolor', 'Species'] = 1
data.loc[data['Species']=='Iris-virginica', 'Species'] = 2

data = data.to_numpy()
data = data[:,1:]

In [246]:
def knn(inp):
    traindata, testdata, trainlabel, testlabel = train_test_split(inp[:,:-1],inp[:,-1],test_size=0.2,random_state=21)
    trainlabel = trainlabel.astype('int')
    testlabel = testlabel.astype('int')
    
    traindata = StandardScaler().fit_transform(traindata)
    testdata = StandardScaler().fit_transform(testdata)

    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(traindata,trainlabel)

    pred = knn.predict(testdata)
    acc = accuracy_score(testlabel,pred)

    return acc*100

In [247]:
def lda(inp):
    c = 3
    label = inp[:,-1]
    tmp = inp[:,:-1]
    glob = np.mean(tmp,axis=0).reshape(-1,1)
    d = tmp.shape[1]
    cent = np.zeros((c,d))

    for i in range(c):
        cent[i] = np.mean(tmp[label==i], axis=0)

    s_w = np.zeros((tmp.shape[1],tmp.shape[1]))
    s_b = np.zeros((tmp.shape[1],tmp.shape[1]))

    for j in range(c):
        s_j = np.zeros((tmp.shape[1],tmp.shape[1]))
        for row in tmp[label==j]:
            row = row.reshape(-1,1)
            mean = cent[j].reshape(-1,1)
            t = (row-mean).dot((row-mean).T).astype('float64')
            s_j += t
        s_w += s_j
        
        n_j = tmp[label==j].shape[0]
        col = cent[i].reshape(-1,1)
        x = (float(n_j)/tmp.shape[0]) * ((col-glob).dot((col-glob).T)).astype('float64')
        s_b += x

    evals,evecs = np.linalg.eig(np.linalg.inv(s_w).dot(s_b))

    sel = np.argsort(evals)[::-1]
    evals = np.sort(evals)[::-1]
    evecs = evecs[:,sel]

    vk = evecs[:,0]
    y = tmp.dot(vk)

    return np.column_stack((y,label))

In [248]:
print(knn(data))

90.0


In [249]:
newdata = lda(data)
print(knn(newdata))

93.33333333333333
